
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src=https://raw.githubusercontent.com/aestaire/ml_workshop/refs/heads/main/files/images/hands-on.png>
</div>


# Demostración de MLOps end-to-end con MLFlow, AutoML y Modelos en Unity Catalog

## Desafíos al llevar un proyecto de ML a producción

Mover un proyecto de ML de un notebook independiente a una canalización de datos de nivel de producción es complejo y requiere múltiples competencias.

Tener un modelo funcionando en un notebook no es suficiente. Necesitamos cubrir todo el ciclo de vida del proyecto de ML y resolver los siguientes desafíos:

* Actualizar los datos con el tiempo (canalización de ingestión de nivel de producción)
* Cómo guardar, compartir y reutilizar características de ML en la organización
* Cómo asegurar que una nueva versión del modelo respete los estándares de calidad y no rompa la canalización
* Gobernanza del modelo: ¿qué está desplegado, cómo se entrenó, por quién y con qué datos?
* Cómo monitorear y reentrenar el modelo...

Además, estos proyectos suelen involucrar a múltiples equipos, creando fricción y posibles silos

* Ingenieros de datos encargados de ingerir, preparar y exponer los datos
* Científicos de datos, expertos en análisis de datos y construcción de modelos de ML
* Ingenieros de ML, que configuran las canalizaciones de infraestructura de ML (similar a DevOps)

Esto tiene un impacto real en el negocio, ralentizando los proyectos y evitando que se desplieguen en producción y generen ROI.

## ¿Qué es MLOps?

MLOps es un conjunto de estándares, herramientas, procesos y metodologías que buscan optimizar el tiempo, la eficiencia y la calidad, asegurando la gobernanza en los proyectos de ML.

MLOps orquesta el ciclo de vida de un proyecto entre los equipos para implementar estas canalizaciones de ML de manera fluida.

Databricks está especialmente posicionado para resolver este desafío con el patrón Lakehouse. No solo reunimos a Ingenieros de Datos, Científicos de Datos e Ingenieros de ML en una plataforma única, sino que también proporcionamos herramientas para orquestar proyectos de ML y acelerar la puesta en producción.

## Recorrido por el proceso de MLOps

En esta demostración, repasaremos algunos pasos comunes en el proceso de MLOps. El resultado de este proceso es un modelo utilizado para alimentar un panel para los interesados del negocio, que incluye:
* preparación de características
* entrenamiento de un modelo para su despliegue
* registro del modelo para que su uso esté gobernado
* validación del modelo en un análisis champion-challenger
* invocación de un modelo de ML entrenado como un UDF de PySpark


<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/mlops/mlops-uc-end2end-0-v2.png?raw=true" width="1200">

<!-- Recopilar datos de uso (vista). Elimínelo para deshabilitar la recopilación o desactive el rastreador durante la instalación. Consulte el README para más detalles.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=1444828305810485&notebook=%2F01-mlops-quickstart%2F00_mlops_end2end_quickstart_presentation&demo_name=mlops-end2end&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fmlops-end2end%2F01-mlops-quickstart%2F00_mlops_end2end_quickstart_presentation&version=1&user_hash=f7ea13a45c991650d8df810431c3e0e2b12887e9ed7e206ee8fb6209bdb2ae82">

### Utilice un cluster Serverless Environment 2 para ejecutar este notebook
Para ejecutar esta demostración, simplemente selecciona el cluster `Serverless` en el menú desplegable.
Comprueba que la versión del cluster serverless es la número 2 <br />



<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src=https://raw.githubusercontent.com/aestaire/ml_workshop/refs/heads/main/files/images/version2-serverless.png>
</div>

### Se ha creado un cluster para esta demostración
Para ejecutar esta demostración, simplemente selecciona el cluster `dbdemos-mlops-end2end` en el menú desplegable ([abrir configuración del clúster](https://e2-demo-field-eng.cloud.databricks.com/#setting/clusters/1013-181446-g00pu2bj/configuration)). <br />
*Nota: Si el cluster fue eliminado después de 30 días, puedes recrearlo con `dbdemos.create_cluster('mlops-end2end')` o reinstalar la demostración: `dbdemos.install('mlops-end2end')`*

En este primer quickstart, cubriremos los fundamentos de MLOps.

- Servir modelos
- Servir características en tiempo real con Online Tables
- Pruebas A/B
- Reentrenamiento automatizado
- Configuración de infraestructura y hooks con Databricks MLOps Stack
- ...

In [0]:
%pip install --quiet mlflow --upgrade


%restart_python

In [0]:
%run ./_resources/00-setup

## Detección de abandono de clientes

Para explorar MLOps, implementaremos un modelo de churn de clientes.

Nuestro equipo de marketing nos pidió crear un Panel que rastree la evolución del riesgo de abandono. Además, necesitamos proporcionar a nuestro equipo de renovaciones una lista diaria de clientes en riesgo de churn para aumentar nuestros ingresos finales.

Nuestro equipo de ingenieros de datos nos proporcionó un conjunto de datos que recopila información sobre nuestra base de clientes, incluyendo información de abandono. Ahí es donde comienza nuestra implementación.

Veamos cómo podemos implementar dicho modelo y proporcionar a nuestros equipos de marketing y renovaciones paneles para rastrear y analizar nuestra predicción de churn.

Finalmente, podrás construir un Panel completo de DBSQL de Churn que contenga toda la información de clientes y abandono, ¡y también iniciar un espacio Genie para hacer cualquier pregunta usando lenguaje natural!

In [0]:
telcoDF = spark.table("mlops_churn_bronze_customers")
display(telcoDF)